In [1]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display

In [26]:
class Region:
    cons = pd.read_excel('IRCM - Consolidation.xlsx', sheet_name='Sheet1')
    def __init__(self, region):
        self.region = region
    
    def check_new_bans(self):
        map_file = pd.read_csv('{}-Customer-contact.csv'.format(self.region))
        map_file.columns = map_file.columns.str.strip()
        Region.cons.columns=Region.cons.columns.str.strip()
        new_bans=[]
        for i in Region.cons.loc[Region.cons.Admininistration == self.region, 'BAN #'].unique():
            if i not in map_file["BAN #"].unique():
                new_bans.append(i)
        return new_bans
    
    def view_new_bans(self):
        map_file = pd.read_csv('{}-Customer-contact.csv'.format(self.region))
        lst_col=copy.deepcopy(map_file.columns).to_list()
        #Adjust the name of one of the columns
        for i, v in enumerate(lst_col):
            if v == 'Current Contact Person ID':
                lst_col[i] = 'Contact Person ID'
        #Generate the new df (with the new BANs)        
        add_df=Region.cons.loc[Region.cons["BAN #"].isin(self.check_new_bans()), lst_col].reset_index(drop=True)
        add_df.columns=map_file.columns
        return add_df

    
    def update_map_file(self):
        map_file = pd.read_csv('{}-Customer-contact.csv'.format(self.region))
        new_map_file = pd.concat([map_file, self.view_new_bans()], axis=0).reset_index(drop=True)
        new_map_file.sort_values(by=['Current Contact Person ID'], na_position='last', inplace=True) 
        return new_map_file.to_csv('{}-Customer-contact.csv'.format(self.region), index=False)
    
     
    def check_file(self):
        map_file = pd.read_csv('{}-Customer-contact.csv'.format(self.region))
        #check not null in one of the requested columns
        cols_to_check1 = ['Customer Number', 'Entity', 'Current Contact Person ID', 'Contact person email address',
                          'Customer Name', 'BAN #', 'Country Code', 'Language Code',
                          'Collector email address', 'Region', 'Workflow']
        cols_to_check2 = ['Contact person email address', 'Customer Name', 'Country Code', 'Language Code','Collector email address',
                          'Account Manager','Region','Phone Number','Workflow']

        if map_file.loc[:, cols_to_check1].isna().sum().sum() > 0:
            series_na = pd.Series(np.nan for i in range(map_file.shape[0]))
            for col in cols_to_check1:
                series_na = series_na | map_file[col].isna()
            print('Missing value in one of the columns')
            miss_values = map_file[series_na]
            display(miss_values)    
        else:
            key = ['Customer Number', 'Entity', 'Current Contact Person ID']
            map_file['key'] = map_file['Customer Number'].map(str)+ \
                              map_file['Entity']+ \
                              map_file['Current Contact Person ID'].map(str)
            map_file['cols_to_check2'] = ""
            for col in cols_to_check2:
                map_file['cols_to_check2'] = map_file['cols_to_check2'].str.cat(map_file[col], na_rep="")
            map_file['key_cols_to_check2'] = map_file['key'].str.cat(map_file['cols_to_check2'], sep=";")
            unique_longstrings = map_file['key_cols_to_check2'].unique()
            extracted_keys = [string.split(";")[0] for string in unique_longstrings]
            # Count the number of occurrencies of each key
            dic={}
            for i in extracted_keys:
                if i not in dic:
                        dic[i]=1
                else:
                        dic[i]+=1
            # filter in only keys with more than 1 occurency
            lst_tuples=[item for item in dic.items() if item[1]>1]
            if len(lst_tuples) == 0:
                print("PASSED: no duplicated keys. The Mapping file can be uploaded")
            else:
                print("FAILED: keys below are duplicated. Check out what needs to be fixed: ")
                for i in lst_tuples:
                    display(map_file[map_file['key']==i[0]].drop(['key','cols_to_check2','key_cols_to_check2'], axis=1))   
            
    def add_IDs(self):
        map_file = pd.read_csv('{}-Customer-contact.csv'.format(self.region))
        for index, row in map_file.iterrows():
            if map_file.loc[index, 'Contact person email address'] != map_file.loc[index, 'Contact person email address']:
                continue
            if np.isnan(row['Current Contact Person ID']):
                 #search above
                 df = map_file[['Current Contact Person ID', 'Contact person email address']].dropna()
                 df = df.drop_duplicates()
                 dic = dict(zip(df['Contact person email address'], df['Current Contact Person ID']))
                 if np.isnan(dic.get(str(row['Contact person email address']), np.nan)):
                    map_file.loc[index, ['Current Contact Person ID']] = map_file['Current Contact Person ID'].max(skipna=True)+1
                 else:
                     map_file.loc[index, ['Current Contact Person ID']] = dic.get(str(row['Contact person email address']), np.nan)
        return map_file.to_csv('{}-Customer-contact.csv'.format(self.region), index=False)
    
                   

In [32]:
we = Region('WE')
#we.check_new_bans()
#we.view_new_bans()
#we.update_map_file()
#we.add_IDs()
#we.check_file()